In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

BASE_PATH = "/content/drive/MyDrive"

students = pd.read_csv(f"{BASE_PATH}/students.csv")
courses = pd.read_csv(f"{BASE_PATH}/courses.csv")
faculty = pd.read_csv(f"{BASE_PATH}/faculty.csv")
enrollments = pd.read_csv(f"{BASE_PATH}/enrollments.csv")




In [ ]:
import os
import sqlite3
import pandas as pd

In [ ]:
students.head()

,student_id,first_name,last_name,email,gender,department,year_of_study,enrollment_date
0,1,Stella,Bradley,stella.bradley@student.university.edu,Female,EARTH SCIENCE,1,2025-08-22
1,2,Aurora,Gray,aurora.gray@student.university.edu,Female,CHEMISTRY,4,2022-09-11
2,3,Jayden,Hunt,jayden.hunt@student.university.edu,Male,MECHANICAL,3,2023-08-16
3,4,Roy,Hansen,roy.hansen@student.university.edu,Male,CHEMICAL,1,2025-08-26
4,5,Asher,Carpenter,asher.carpenter@student.university.edu,Male,CSE,3,2023-08-23


In [ ]:
students_schema = """
CREATE TABLE IF NOT EXISTS students (
    student_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    gender VARCHAR(10),
    department VARCHAR(50),
    year_of_study INT,
    enrollment_date DATE
);
"""


In [ ]:
courses.head()

,course_id,course_name,department,credits,semester,faculty_id
0,1,Data Structures and Algorithms,CSE,4,Fall,13
1,2,Machine Learning Fundamentals,CSE,4,Spring,5
2,3,Materials Science Principles,MSE,3,Fall,7
3,4,Thermodynamics,CHEMICAL,4,Spring,21
4,5,Chemical Process Design,CHEMICAL,3,Fall,12


In [ ]:
courses_schema = """
CREATE TABLE IF NOT EXISTS courses (
    course_id INT PRIMARY KEY,
    course_name VARCHAR(100),
    department VARCHAR(50),
    credits INT,
    semester VARCHAR(10),
    faculty_id INT
);
"""


In [ ]:
faculty.head()

,faculty_id,first_name,last_name,email,department,designation
0,1,Audrey,Jacobs,audrey.jacobs@university.edu,CSE,Assistant Professor
1,2,Jerry,Scott,jerry.scott@university.edu,CHEMICAL,Associate Professor
2,3,Jayden,Chen,jayden.chen@university.edu,MSE,Assistant Professor
3,4,Roy,Campbell,roy.campbell@university.edu,MSE,Assistant Professor
4,5,Virginia,Mendez,virginia.mendez@university.edu,CSE,Associate Professor


In [ ]:
faculty_schema = """
CREATE TABLE IF NOT EXISTS faculty (
    faculty_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    department VARCHAR(50),
    designation VARCHAR(50)
);
"""


In [ ]:
enrollments.head()

,enrollment_id,student_id,course_id,enrollment_date,grade
0,1,1,6,2026-01-02,NaN
1,2,1,1,2025-09-04,NaN
2,3,1,4,2025-09-02,NaN
3,4,1,5,2025-08-25,NaN
4,5,2,2,2022-09-20,C


In [ ]:
enrollments_schema = """
CREATE TABLE IF NOT EXISTS enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    enrollment_date DATE,
    grade VARCHAR(5),
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);
"""


In [ ]:
db_name = 'college.db'

if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [ ]:
import sqlite3
import pandas as pd
import os

# -----------------------------
# Expected column dtypes (Pandas)
# -----------------------------
COLUMN_DATA_TYPES = {
    'students': {
        'student_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'gender': 'object',
        'department': 'object',
        'year_of_study': 'int64',
        'enrollment_date': 'datetime64[ns]'
    },
    'courses': {
        'course_id': 'int64',
        'course_name': 'object',
        'department': 'object',
        'credits': 'int64',
        'semester': 'object',
        'faculty_id': 'int64'
    },
    'faculty': {
        'faculty_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'department': 'object',
        'designation': 'object'
    },
    'enrollments': {
        'enrollment_id': 'int64',
        'student_id': 'int64',
        'course_id': 'int64',
        'enrollment_date': 'datetime64[ns]',
        'grade': 'object'
    }
}

# -----------------------------
# Schemas
# students_schema
# courses_schema
# faculty_schema
# enrollments_schema
# -----------------------------

# --- Database setup ---
db_name = 'university.db'
conn = None

try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()


    cursor.execute("PRAGMA foreign_keys = ON;")

    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables (order matters because of foreign keys)
    cursor.execute(faculty_schema)
    cursor.execute(students_schema)
    cursor.execute(courses_schema)
    cursor.execute(enrollments_schema)
    print("Tables 'faculty', 'students', 'courses', and 'enrollments' created successfully.")

    # --- Load data from CSV files into the tables ---

    BASE_PATH = "/content/drive/MyDrive"
    csv_to_table_map = {
        f'{BASE_PATH}/faculty.csv': 'faculty',
        f'{BASE_PATH}/students.csv': 'students',
        f'{BASE_PATH}/courses.csv': 'courses',
        f'{BASE_PATH}/enrollments.csv': 'enrollments'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            df = pd.read_csv(csv_file)

            # Expected schema for the table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # Drop extra columns
            df = df[df.columns.intersection(expected_cols)]

            # Add missing columns as NULL
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # Reorder columns
            df = df[expected_cols]

            # Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert '{col}' to {dtype}. Error: {e}. Leaving as is.")

            # Load into SQLite (replace to avoid duplicates if you rerun)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping '{table_name}'.")

    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: Missing table in COLUMN_DATA_TYPES: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("Database connection closed.")


Database 'university.db' created and connected successfully. ✅
Tables 'faculty', 'students', 'courses', and 'enrollments' created successfully.

Processing '/content/drive/MyDrive/faculty.csv' for table 'faculty'...
  -> Data from '/content/drive/MyDrive/faculty.csv' loaded into 'faculty' successfully.

Processing '/content/drive/MyDrive/students.csv' for table 'students'...
  -> Data from '/content/drive/MyDrive/students.csv' loaded into 'students' successfully.

Processing '/content/drive/MyDrive/courses.csv' for table 'courses'...
  -> Data from '/content/drive/MyDrive/courses.csv' loaded into 'courses' successfully.

Processing '/content/drive/MyDrive/enrollments.csv' for table 'enrollments'...
  -> Data from '/content/drive/MyDrive/enrollments.csv' loaded into 'enrollments' successfully.

Data committed to the database successfully. 🎉
Database connection closed.


In [ ]:
!pip install google-genai

In [ ]:
from google import genai
from google.colab import userdata

In [ ]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
prompt = """

###ROLE###
You are a highly skilled Text-to-SQL translator with expertise in SQL syntax, database schema interpretation, and natural language understanding. You generate syntactically correct and semantically accurate SQL queries based on user input and a given database schema.

###CONTEXT###
The user is working with a relational database for a university. The database includes four main tables: `students`, `faculty`, `courses`, and `enrollments`. The goal is to allow users to input natural language queries (in English), and have the model return equivalent SQL statements that accurately extract the requested data using the given schema.

Here is the full schema:

**Students Table**
```sql
CREATE TABLE IF NOT EXISTS students (
    student_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    gender VARCHAR(10),
    department VARCHAR(50),
    year_of_study INT,
    enrollment_date DATE
);
````

**Faculty Table**

```sql
CREATE TABLE IF NOT EXISTS faculty (
    faculty_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    department VARCHAR(50),
    designation VARCHAR(50)
);
```

**Courses Table**

```sql
CREATE TABLE IF NOT EXISTS courses (
    course_id INT PRIMARY KEY,
    course_name VARCHAR(100),
    department VARCHAR(50),
    credits INT,
    semester VARCHAR(10),
    faculty_id INT
);
```

**Enrollments Table**

```sql
CREATE TABLE IF NOT EXISTS enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    enrollment_date DATE,
    grade VARCHAR(5),
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);
```

###TASK###
Your task is to:

1. Read a natural language query about the university data.
2. Interpret the user's intent based on the schema provided.
3. Generate a valid SQL SELECT query that returns the expected result.
4. Ensure correct table joins, column selection, filtering, and grouping as necessary.
5. Handle aggregate functions (e.g., COUNT, AVG, SUM) where appropriate.
6. Disambiguate user terms based on schema details, for example:
  "students" → students
  "professors/faculty" → faculty
  "courses/classes" → courses
  "enrolled/took/registered" → enrollments
  "instructor of a course" → courses.faculty_id joined to faculty.faculty_id
  "students in a course" → join students ↔ enrollments ↔ courses
  "department" may refer to students.department, courses.department, or faculty.department depending on context

###CONSTRAINTS###

* Only return a valid SQL query as output — no explanations or extra text.
* The user is using sqllite database - respond with correct and valid sqllite syntax
* Use aliases (`AS`) for column names only when the original name is ambiguous.
* Do not create or modify tables.
* Do not assume the existence of tables or columns not provided in the schema.
* Avoid subqueries unless absolutely necessary for correctness or performance.
* Prefer readability: indent joins and clauses properly.

###EXAMPLES###
**Input:** "Show me the names and emails of students from the CSE department in 3rd year."
**Output:**

```sql
SELECT first_name, last_name, email
FROM students
WHERE department = 'CSE' AND year_of_study = 3;
```

**Input:** "List the top 5 courses with the highest credits."
**Output:**

```sql
SELECT course_name, department, credits
FROM courses
ORDER BY credits DESC
LIMIT 5;
```

**Input:** "How many students are enrolled in each course?"
**Output:**

```sql
SELECT
    c.course_name,
    COUNT(e.student_id) AS total_students
FROM courses c
LEFT JOIN enrollments e
    ON c.course_id = e.course_id
GROUP BY c.course_id, c.course_name
ORDER BY total_students DESC;
```

**Input:** "Show the faculty name teaching 'Machine Learning Fundamentals'."
**Output:**

```sql
SELECT f.first_name, f.last_name
FROM courses c
JOIN faculty f
    ON c.faculty_id = f.faculty_id
WHERE c.course_name = 'Machine Learning Fundamentals';
```

###OUTPUT FORMAT###
Return only the sqllite SQL query as a code block using triple backticks and the `sql` language tag, like this:

```sql
-- Your SQL query here
```
"""

In [ ]:
import json
def get_sql_query_via_gemini(genai_client, prompt, user_query):

  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = response.text.replace('```sql', '').replace('```', '')

  return output


In [ ]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='university.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [ ]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query_via_gemini(genai_client, prompt, user_query)
  results = execute_query(output)
  return results

In [ ]:
text2sql(genai_client, prompt, "How many students are enrolled in each course?")

Input Token Count: 1134
Thoughts Token Count: 106
Output Token Count: 74
Total Token Count: 1314

Executing query on 'university.db':

SELECT
    c.course_name,
    COUNT(e.student_id) AS total_students
FROM courses c
LEFT JOIN enrollments e
    ON c.course_id = e.course_id
GROUP BY c.course_id, c.course_name
ORDER BY total_students DESC;

Query executed successfully.


,course_name,total_students
0,Quantum Mechanics,192
1,Classical Mechanics,190
2,Machine Learning Fundamentals,185
3,Materials Science Principles,185
4,Chemical Process Design,179
5,Organic Chemistry,178
6,Analytical Chemistry,177
7,Fluid Mechanics,171
8,Aerodynamics,170
9,Data Structures and Algorithms,165
